# 1. [20 pts] List three popular RNN types, then briefly compare and contrast their features and applications.

Three popular RNN types are Elman RNNs, LSTM RNNs and GRU RNNs. Elman RNNs 

# 2. [20 pts] Explain why the LSTM RNN performed superbly compared to the remainder of the classifiers in the module.

# 3. [20 pts] From the surname dataset, using only two languages {English and Scottish}, build a classifier to distinguish between them. Report 10-fold cross validation performance.

# 4. [20 pts] Set the WEIGHTS to None and report the classification performance of (3.). Explain what caused the performance drop?

In [50]:
import os
import pandas as pd
import csv
folder_path=['./surnames/English.txt','./surnames/Scottish.txt']

def folder_parse(path):
    names=[]

    with open(path, 'r', encoding="utf-8") as f:
        for line in f:
            name = line.strip()  # Remove leading/trailing whitespaces
            #print(list(name))
            names.append(len(name))
    return names
for folder in folder_path:
    stats=folder_parse(folder)
    print(len(stats),max(stats))

3668 14
100 10


In [42]:
from os import listdir, path
import numpy as np
PATH_DATA='./surnames'
LANGS=['English','Scottish']
SEQ_SIZE=14

In [43]:
# Letter index 0 is the padding value, i.e. padding to fill up the vector to SEQ_SIZE, necessary for batched
# Note that eventually we will use torch Tensor to represent these fixed length sequences
LetterVocabulary, LetterVocabularyIndex, Index2Voc, Sequences = {' ':0}, 1, {0:' '}, {}

for fn in sorted([_ for _ in listdir(PATH_DATA) if _.endswith('.txt')]):
    lang, seqs = path.splitext(path.basename(fn))[0], []

    if lang not in LANGS:  # test case
        continue

    with open(path.join(PATH_DATA, fn), 'r', encoding="utf8") as fin:
        for row in fin.read().splitlines():
            seq = np.zeros(SEQ_SIZE, dtype=np.int32)
            for i, letter in enumerate(row.lower()):  # Convert the surname to lower case
#            for i, letter in enumerate(row):
                if i < SEQ_SIZE:
                    if letter not in LetterVocabulary:
                        LetterVocabulary[letter] = LetterVocabularyIndex
                        Index2Voc[LetterVocabularyIndex] = letter
                        LetterVocabularyIndex += 1
                    seq[i] = LetterVocabulary[letter]
            seqs += [seq]
    Sequences[lang] = seqs

In [46]:
# Sanity check
def print_names(_lang, _k):
    print(''.join([Index2Voc[c] for c in Sequences[_lang][_k]]), Sequences[_lang][_k]) if _lang in LANGS else None

# Some examples
print_names('English', 1)
print_names('Scottish', 2)

abbey          [1 2 2 4 5 0 0 0 0 0 0 0 0 0]
wilson         [18  9 10  3  6 16  0  0  0  0  0  0  0  0]


In [49]:
# Sanity
N = sum([len(Sequences[_]) for _ in Sequences])
T = Sequences[''][0].shape[0]
C = len(np.unique(Sequences.keys())[0])
print(N, T, C)

3768 14 2


In [51]:
import itertools

# Pool all sequences and all languages
Seqs = [Sequences[LANGS[_]] for _ in range(C)]
Seqs = list(itertools.chain(*Seqs))

# Number of features is the number of unique characters
M = np.max(Seqs)
print(f'M= {M}')

M= 26


In [53]:
import torch
# Apriori class balance, i.e. inverse probability of the class
nk = np.array([len(Sequences[LANGS[_]]) for _ in range(C)], dtype=np.float32)
nk = (N/nk)
nk = nk/nk.sum()

# Class weights, inverse apriori probability
WEIGHTS = torch.tensor(nk, dtype=torch.float32)

print(WEIGHTS)

tensor([0.0265, 0.9735])


# 4. [20 pts] Set the WEIGHTS to None and report the classification performance of (3.). Explain what caused the performance drop?